# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    # 模擬網頁下拉
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 等待一段時間讓網頁更新
    time.sleep(SCROLL_PAUSE_TIME)

    # 檢視目前網頁底端
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    print("new_height:",new_height)
    last_height = new_height
print("last_height:",last_height)

new_height: 9156
new_height: 11492
new_height: 13784
new_height: 16102
new_height: 18452
new_height: 20803
new_height: 23150
new_height: 24543
new_height: 24593
last_height: 24593


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


0名嚴重特殊傳染性肺炎（武漢肺炎）患者，10名個案狀況穩定，且有1人採檢都是陰性，可望近日出院，另9名患者也都是輕症，並赴中港澳旅遊回台者，都要進行居家檢疫 14 天。
官員指出，中國面對武漢肺炎的疫情，為了防疫，同樣有對於人流、物流的管制。對於台商而言，會出現部分區域的產能、供應鏈整合，以及終端消費市場受影響的效應。
針對疫情恐衝擊經濟與產業前景，經濟部也祭出三招，協助台商與各產業因應：
▲針對疫情恐衝擊經濟與產業前景，經濟部祭出三招，協助台商與各產業因應。（圖／經濟部提供）
----------------------------------------------------------------------

[65] 口罩出售實名制明日起跑　財政部定調：特約藥局免開發票

▲藥局明日起開賣實名制口罩，可以不用開發票。（圖／記者黃軍瑋攝）
記者吳靜君／台北報導
因應武漢肺炎疫情，政府實施實名制購買口罩政策明（6）日上路，政府規範每一人7天之內只能買2片；財政部今（5）日核釋，健保特約藥局因應武漢肺炎疫情，在嚴重特殊傳染性肺炎中央流行疫情指揮中心成立期間，受政府機關委託，依政府機關規定價格代售徵用口罩取得的款項，全數交該委託機關解繳公庫者，依傳染病防治法第55條規定免課徵營業稅，免開立統一發票。
財政部說明，嚴重特殊傳染性肺炎持續升溫，配合疫情指揮中心宣布，自109年2月6日起口罩販售採實名制，由郵局配送政府徵用的口罩至健保特約藥局委託代售。
依傳染病防治法第55條規予以釋明，藥局是依政府機關規定價格代售徵用或配銷之防疫物資（口罩），其出售收入全數交該委託機關解繳公庫者，免課徵營業稅可以免營業稅，以利徵納雙方遵循。
外界也好奇，超商也代賣口罩一次三片，卻有開立統一發票，財政部解釋，因為超商是屬於營業人，銷售商品必須要開立發票，但是依據傳染病防治法規範，也可以免營業稅；至於藥局是執行業務，非營業人而且代政府銷售口罩，也沒有收取費用、利潤，所以也沒有所得，所以也免所得稅。
----------------------------------------------------------------------

[66] 企業清償能力受武漢肺炎疫情影響　央行：票信依規定從寬處理

▲ 武漢肺炎疫情升溫，央行對受影響產業之票信從寬處理。（示意圖／路透）
記者李瑞